### Tool Calling

Tool calling is the capability to give the model access to external functionalities and APIs. In this notebook we will be walking through the concept and best practices for using this capability with Nova models.

### Defining the Schema

When we refer to a "tool" we are referring to a function that will execute actual code. To provide the details about this function, you will provide a tool configuration to the model. This tool configuration will contain details such as the name, description and details about the parameters. 

You can imagine a calculator tool that might be defined as:
```python
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "calculator", # Name of the tool
                "description": "A calculator tool that can execute a math equation",
                "inputSchema": {
                    "json": { 
                        "type": "object", # The top level schema MUST have a type of "object", properities and required keys. No other fields are allowed at this level
                        "properties": {
                            "equation": { # The name of the parameter
                                "type": "string", # parameter type: string/int/etc
                                "description": "The full equation to evaluate" # Helpful description of the parameter
                            }
                        },
                        "required": [ # List of all required parameters
                            "equation"
                        ]
                    }
                }
            }
        }
    ]
}
```

or a retriever tool that allows you to search SEC filings

```python
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "secRetriever", 
                "description": "A retriever that can access SEC filings from a database",
                "inputSchema": {
                    "json": { 
                        "type": "object", # The top level schema MUST have a type of "object", properities and required keys. No other fields are allowed at this level
                        "properties": {
                            "query": { 
                                "type": "string", 
                                "description": "The full query to search for" 
                            },
                            "ticker": { 
                                "type": "string", 
                                "description": "The stock ticker of the company"
                            },
                            "year": { 
                                "type": "string", 
                                "description": "The relevant year of the filings"
                            }
                        },
                        "required": [ # Note that year is not provided, this indicates it's an optional parameter
                            "query",
                            "ticker"
                        ]
                    }
                }
            }
        }
    ]
}
```

or a multiplication tool that takes two integers and multiplies them

```python
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "multiply",
                "description": "Multiplies two numbers together",
                "inputSchema": {
                    "json": { 
                        "type": "object", # The top level schema MUST have a type of "object", properities and required keys. No other fields are allowed at this level
                        "properties": {
                            "int1": { 
                                "type": "integer", 
                                "description": "The first number to multiply" 
                            },
                            "int2": { 
                                "type": "integer", 
                                "description": "The second number to multiply"
                            }
                        },
                        "required": [
                            "int1",
                            "int2"
                        ]
                    }
                }
            }
        }
    ]
}

```


### Provide the Tool to Nova
In these examples we will be using the converse API and you can pass the tools to the model through the toolConfig parameter on the model. When we are utilizing tool calling - we recommend taking advantage of "greedy decoding" parameters. With Nova this is done by setting the temperature, topP and topK to 1.

Starting with the calculator:


In [ ]:
import boto3

LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"

client = boto3.client("bedrock-runtime", region_name="us-east-1")

system = [{"text": "You should only use the tools provided and not your own knowledge"}]


messages = [{"role": "user", "content": [{"text": "What is 2+2"}]}]

tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "calculator",
                "description": "A calculator tool that can execute a math equation",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "equation": {
                                "type": "string",
                                "description": "The full equation to evaluate",
                            }
                        },
                        "required": ["equation"],
                    }
                },
            }
        }
    ]
}

inf_params = {"maxTokens": 300, "topP": 1, "temperature": 1}

initial_response = client.converse(
    modelId=LITE_MODEL_ID,
    system=system,
    messages=messages,
    inferenceConfig=inf_params,
    additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
    toolConfig=tool_config,
)
tool_use = next(
    block["toolUse"]
    for block in initial_response["output"]["message"]["content"]
    if "toolUse" in block
)

print(tool_use)

The retriever:

In [ ]:
import boto3

LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"

client = boto3.client("bedrock-runtime", region_name="us-east-1")

system = [{"text": "You should only use the tools provided and not your own knowledge"}]


messages = [
    {
        "role": "user",
        "content": [{"text": "What was Amazon's reported revenue in 2023"}],
    }
]

tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "secRetriever",
                "description": "A retriever that can access SEC filings from a database",
                "inputSchema": {
                    "json": {
                        "type": "object",  # The top level schema MUST have a type of "object", properities and required keys. No other fields are allowed at this level
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "The full query to search for",
                            },
                            "ticker": {
                                "type": "string",
                                "description": "The stock ticker of the company",
                            },
                            "year": {
                                "type": "string",
                                "description": "The relevant year of the filings",
                            },
                        },
                        "required": [  # Note that year is not provided, this indicates it's an optional parameter
                            "query",
                            "ticker",
                        ],
                    }
                },
            }
        }
    ]
}

inf_params = {"maxTokens": 300, "topP": 1, "temperature": 1}

initial_response = client.converse(
    modelId=LITE_MODEL_ID,
    system=system,
    messages=messages,
    inferenceConfig=inf_params,
    additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
    toolConfig=tool_config,
)
tool_use = next(
    block["toolUse"]
    for block in initial_response["output"]["message"]["content"]
    if "toolUse" in block
)

print(tool_use)

And finally the multiplication tool

In [ ]:
import boto3

LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"

client = boto3.client("bedrock-runtime", region_name="us-east-1")

messages = [{"role": "user", "content": [{"text": "What is 2*2"}]}]

system = [{"text": "You should only use the tools provided and not your own knowledge"}]

tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "multiply",
                "description": "Multiplies two numbers together",
                "inputSchema": {
                    "json": {
                        "type": "object",  # The top level schema MUST have a type of "object", properities and required keys. No other fields are allowed at this level
                        "properties": {
                            "int1": {
                                "type": "int",
                                "description": "The first number to multiply",
                            },
                            "int2": {
                                "type": "int",
                                "description": "The second number to multiply",
                            },
                        },
                        "required": ["int1", "int2"],
                    }
                },
            }
        }
    ]
}
inf_params = {"maxTokens": 300, "topP": 1, "temperature": 1}

initial_response = client.converse(
    modelId=LITE_MODEL_ID,
    system=system,
    messages=messages,
    inferenceConfig=inf_params,
    additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
    toolConfig=tool_config,
)
tool_use = next(
    block["toolUse"]
    for block in initial_response["output"]["message"]["content"]
    if "toolUse" in block
)

print(tool_use)

### Using Tool Choice

With Amazon Nova models, you can utiize the "Tool Choice" API parameter. By using the tool choice parameter, you can control the behavior of the model when selecting a tool. There are three options available:

**Tool**: The specified tool will be called once

```python
"toolChoice": {
   "name": { "tool" : <tool_name> }
}
```

**Any**: One of the provided tools will be called at least once

```python
"toolChoice": {
   "any": {}
}
```

**Auto**: The model will decide whether to call a tool. Multiple tools can be called if required

```python
"toolChoice": {
   "auto": {}
}
```

#### Tool Choice - Tool

The option of "tool" is common in use cases such as structured output where we might want to enforce the model to call the same tool every time.

In [ ]:
import boto3

with open("media/nutritional_benifits.png", "rb") as media_file:
    binary_data = media_file.read()


tool_config = {
    "toolChoice": {"tool": {"name": "print_nutrition_info"}},
    "tools": [
        {
            "toolSpec": {
                "name": "print_nutrition_info",
                "description": "Extracts nutrition information from an image of a nutrition label",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "calories": {
                                "type": "integer",
                                "description": "The number of calories per serving",
                            },
                            "total_fat": {
                                "type": "integer",
                                "description": "The amount of total fat in grams per serving",
                            },
                            "cholesterol": {
                                "type": "integer",
                                "description": "The amount of cholesterol in milligrams per serving",
                            },
                            "total_carbs": {
                                "type": "integer",
                                "description": "The amount of total carbohydrates in grams per serving",
                            },
                            "protein": {
                                "type": "integer",
                                "description": "The amount of protein in grams per serving",
                            },
                        },
                        "required": [
                            "calories",
                            "total_fat",
                            "cholesterol",
                            "total_carbs",
                            "protein",
                        ],
                    }
                },
            }
        },
    ],
}

messages = [
    {
        "role": "user",
        "content": [
            {
                "image": {
                    "format": "png",
                    "source": {"bytes": binary_data},
                }
            },
            {
                "text": "Please print the nutrition information from this nutrition label image"
            },
        ],
    }
]

inf_params = {"topP": 1, "temperature": 1}

client = boto3.client("bedrock-runtime", region_name="us-east-1")

response = client.converse(
    modelId="us.amazon.nova-lite-v1:0",
    messages=messages,
    toolConfig=tool_config,
    inferenceConfig=inf_params,
    additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
)

print(
    next(
        block["toolUse"]
        for block in response["output"]["message"]["content"]
        if "toolUse" in block
    )
)

#### Tool Choice - Any

Some use cases will require that a tool is always called, no matter the context of the user query. 

In [ ]:
import boto3

tool_config = {
    "toolChoice": {"any": {}},
    "tools": [
        {
            "toolSpec": {
                "name": "get_the_weather",
                "description": "API to get the current weather",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "city": {
                                "type": "string",
                                "description": "The city to get the weather for",
                            },
                        },
                        "required": ["city"],
                    }
                },
            }
        },
        {
            "toolSpec": {
                "name": "follow_up_question",
                "description": "Ask a follow up question to the user",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "question": {
                                "type": "string",
                                "description": "Question to ask the user",
                            },
                        },
                        "required": ["question"],
                    }
                },
            }
        },
    ],
}

system = [
    {
        "text": "You can get the current weather for the user, if you need more information use the follow_up_question tool"
    }
]


messages = [
    {
        "role": "user",
        "content": [
            {"text": "Can you get the weather"},
        ],
    }
]

inf_params = {"topP": 1, "temperature": 1}

client = boto3.client("bedrock-runtime", region_name="us-east-1")

response = client.converse(
    modelId="us.amazon.nova-lite-v1:0",
    system=system,
    messages=messages,
    toolConfig=tool_config,
    inferenceConfig=inf_params,
    additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
)

print(
    next(
        block["toolUse"]
        for block in response["output"]["message"]["content"]
        if "toolUse" in block
    )
)

#### Tool Choice - Auto

For use cases where a tool isn't always required - you can set the tool choice to auto. This is the default behavior and will leave the tool selection completely up to the model.

In [ ]:
import boto3
import json
import re

tool_config = {
    "toolChoice": {"auto": {}},
    "tools": [
        {
            "toolSpec": {
                "name": "search",
                "description": "API that provides access to the internet",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "Query to search by",
                            },
                        },
                        "required": ["query"],
                    }
                },
            }
        }
    ],
}

messages = [
    {
        "role": "user",
        "content": [
            {"text": "Who was in the cast of Wicked"},
        ],
    }
]

system = [{"text": "You are a helpful chatbot. You can use a tool if necessary"}]

inf_params = {"topP": 1, "temperature": 1}

client = boto3.client("bedrock-runtime", region_name="us-east-1")

response = client.converse(
    modelId="us.amazon.nova-micro-v1:0",
    messages=messages,
    toolConfig=tool_config,
    inferenceConfig=inf_params,
    additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
)

print(response["output"]["message"]["content"])